***In this notebook we preprocess the Eurovision dataset from 1975-2019***

In [3]:
#!pip install openpyxl
import pandas as pd
df = pd.read_excel('eurovision_song_contest_1975_2019.xlsx')
df = df.drop(df[df['(semi-) final'] != 'f'].index)
df = df.drop(df[df['Year'] < 2017].index)
df = df.drop('Duplicate', axis = 1)
df = df.drop('Edition', axis = 1)
df = df.drop('(semi-) final', axis = 1)
df = df.rename({'Points      ': 'Points'}, axis='columns')


In [4]:
df_2019 = df[df['Year'] == 2019].reset_index()
df_2018 = df[df['Year'] == 2018].reset_index()
df_2017 = df[df['Year'] == 2017].reset_index()

In [5]:
def sum_total(df):
    vocab = {}
    for country in df['To country'].unique():
        summ = 0
        for j in range(0, df.shape[0]):
            if df.iloc[j]['To country'] == country:
                summ = summ + df.iloc[j].values[5] #points
        vocab[country] = summ
    return vocab
# Extract datas into vocabularies 
voc_2019 = {k: v for k, v in sorted(sum_total(df_2019).items(),reverse = True, key=lambda item: item[1])}
voc_2018 = {k: v for k, v in sorted(sum_total(df_2018).items(),reverse = True, key=lambda item: item[1])}
voc_2017 = {k: v for k, v in sorted(sum_total(df_2017).items(),reverse = True, key=lambda item: item[1])}

In [6]:
def append_tot(df, voc):
    new_l = []
    for i in range(0, df.shape[0]):
        new_l.append(voc[df.iloc[i]['To country']])
    df['Tot'] = new_l
    return df
# Extract dataframe for each computed vocabulary
df_2019 = append_tot(df_2019, voc_2019)
df_2018 = append_tot(df_2018, voc_2018)
df_2017 = append_tot(df_2017, voc_2017)


In [12]:
def merge_j_t(df_year):
  df_t = df_year[df_year['Jury or Televoting'] == 'T'].reset_index()
  df_t = df_t.drop({'index', 'level_0'}, axis=1)
  df_j = df_year[df_year['Jury or Televoting'] == 'J'].reset_index()
  df_j = df_j.drop({'index', 'level_0'}, axis=1)
  df_final = df_t.copy()
  for i in range(0, df_t.shape[0]):
    df_final.at[i,'Points']= df_t.iloc[i][4] + df_j.iloc[i][4]
  return df_final
df_final_2017 = merge_j_t(df_2017)
df_final_2018 = merge_j_t(df_2018)
df_final_2019 = merge_j_t(df_2019)

df_final_2017 = df_final_2017.drop(columns=['Jury or Televoting','Year'],axis=1).head()
df_final_2018 = df_final_2018.drop(columns=['Jury or Televoting','Year'],axis=1).head()
df_final_2019 = df_final_2019.drop(columns=['Jury or Televoting','Year'],axis=1).head()

***Now the dataframes are in the same 'melted' form of the other notebook***